<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN04/4.3-AutoDiff_Demo.ipynb)


In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
%pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# AutoGrad-Demo

Wir werden uns drei verschiedene Methoden zur Berechnung von Gradienten ansehen, die alle auf demselben Grundprinzip beruhen (Backprop). 

1. Die erste Methode besteht darin, eine Funktion und ihre Ableitungen manuell nach den Regeln von Backprop zu kodieren.
2. Die zweite Methode ist die automatische Differenzierung mit einem Tool namens
[autograd](https://github.com/HIPS/autograd).
3. Die dritte Methode ist die Verwendung von [Pytorch Lightning](https://lightning.ai/docs/pytorch/stable/)



### Mehr über ``autograd``

[autograd](https://github.com/HIPS/autograd) ist ein Python-Paket für **algorithmische Differenzierung**. Es erlaubt die automatische Berechnung der Ableitung von Funktionen, die in (fast) nativem Code geschrieben sind. Das macht die Berechnung von Ableitungen sehr einfach. Unter der Haube verwendet es auch Reverse Mode Autodiff (Backprop).


# Backpropagation: Beispiel 1

Wir werden drei verschiedene Möglichkeiten zur Berechnung des Gradienten von

$$f(x,y,z) = (2x + y)\cdot z$$

am Punkt $(x,y,z)=(1,2,3)$

### Manuelle Backpropagation

In [ ]:
!pip install requests
!pip install autograd
# !git clone https://github.com/dsgiitr/d2l-pytorch.git
# Homepage
# https://d2l.ai/


import numpy as np

# Backprop example

# Compute f(x,y,z) = (2*x+y)*z
x = 1.0
y = 2.0
z = 3.0

# Forward pass
q = 2.0 * x + y  # Node 1
f = q * z  # Node 2

# Backward pass
f_bar = 1
q_bar = z * f_bar  # Node 2 input
z_bar = q * f_bar  # Node 2 input
x_bar = 2 * q_bar  # Node 1 input
y_bar = 1 * q_bar  # Node 1 input

grad = np.array([x_bar, y_bar, z_bar])

print(f)
print(grad)


### Autograd

In [ ]:
# %pip install autograd

import autograd.numpy as np  # Thinly wrapped version of numpy
from autograd import grad


def f(args):
    x, y, z = args
    return (2 * x + y) * z


f_grad = grad(f)  # magic: returns a function that computes the gradient of f

x = 1.0
y = 2.0
z = 3.0

print(f([x, y, z]))
print(f_grad([x, y, z]))


# Backpropagation: Beispiel 2

Hier ist ein etwas komplizierteres Beispiel:

$$f(x) = 10\cdot \exp(\sin(x)) + \cos^2(x)$$


Berechnen Sie mit manuell mit dem Backpropagation Algorithmus den Gradienten am Punkt $(x,y,z)=(1,2,3)$

### Manual backprop

In [ ]:
import numpy as np

# Backprop example
# f(x) = 10*np.exp(np.sin(x)) + np.cos(x)**2

# Forward pass
x = 1000
a = np.sin(x)  # Node 1
b = np.cos(x)  # Node 1
c = b**2  # Node 3
d = np.exp(a)  # Node 4
f = 10 * d + c  # Node 5 (final output)

# Backward pass
f_bar = 1
d_bar = 10 * f_bar  # Node 5 input
c_bar = 1 * f_bar  # Node 5 input
a_bar = np.exp(a) * d_bar  # Node 4 input
b_bar = 2 * b * c_bar  # Node 3 input
x_bar = np.cos(x) * a_bar - np.sin(x) * b_bar  # Node 2 and 1 input

print(f, x_bar)


### Autograd

In [ ]:
import autograd.numpy as np  # Thinly wrapped version of numpy
from autograd import grad


def f(args):
    x = args
    return 10 * np.exp(np.sin(x)) + np.cos(x) ** 2


f_grad = grad(f)

x = 1000.0

print(f([x]))
print(f_grad([x]))


# Pytorch Lightning

In [ ]:
# %pip install pytorch-lightning
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.autograd import grad

# PyTorch Lightning Backpropagation und AutoDiff Beispiele

class BackpropModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # Initialisiere Tensoren mit requires_grad=True, um Berechnungen zu verfolgen
        self.x = torch.tensor(1.0, requires_grad=True)
        self.y = torch.tensor(2.0, requires_grad=True)
        self.z = torch.tensor(3.0, requires_grad=True)

    def forward(self):
        # Berechne q = 2*x + y
        q = 2 * self.x + self.y  # Knoten 1
        # Berechne f = q * z
        f = q * self.z  # Knoten 2
        return f

    def compute_gradients(self):
        # Führe den Vorwärtsdurchlauf durch
        f = self.forward()
        # Berechne Gradienten (Rückwärtsdurchlauf)
        f.backward()
        # Gebe die Gradienten von x, y und z zurück
        return self.x.grad, self.y.grad, self.z.grad

# PyTorch Lightning Version der komplexen Funktion f(x)

class ComplexFunction(pl.LightningModule):
    def __init__(self):
        super().__init__()
        # Initialisiere Tensor mit requires_grad=True, um Berechnungen zu verfolgen
        self.x = torch.tensor(1000.0, requires_grad=True)

    def forward(self):
        # Berechne a = sin(x)
        a = torch.sin(self.x)  # Knoten 1
        # Berechne b = cos(x)
        b = torch.cos(self.x)  # Knoten 2
        # Berechne c = b^2
        c = b**2  # Knoten 3
        # Berechne d = exp(a)
        d = torch.exp(a)  # Knoten 4
        # Berechne f = 10*d + c
        f = 10 * d + c  # Knoten 5 (Endergebnis)
        return f

    def compute_gradients(self):
        # Führe den Vorwärtsdurchlauf durch
        f = self.forward()
        # Berechne Gradienten (Rückwärtsdurchlauf)
        f.backward()
        # Gebe den Gradienten von x zurück
        return self.x.grad


# Instanziiere und berechne Gradienten
model = BackpropModel()
grads = model.compute_gradients()
print("Funktionsausgabe:", model.forward().item())
print("Gradienten:", grads)

# Führe das Beispiel der komplexen Funktion aus
complex_model = ComplexFunction()
x_grad = complex_model.compute_gradients()
print("Ausgabe der komplexen Funktion:", complex_model.forward().item())
print("Gradient der komplexen Funktion:", x_grad.item())
